In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model

import plotly.express as px # for interactive plots

#Load the Solar Data

In [ ]:
'''
DATA_DIR_SOLAR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance"
hist_window=18;forecast_window=18
for y in range(2017,2022):
  df = pd.read_csv(f"{DATA_DIR_SOLAR}/solar_{y}.csv",header=2)
  #print(df.head(10))

  #enforce 5 m and Interpolate
  df['DATE'] = df['Month'].astype(str)+ '-' +df['Day'].astype(str)+ '-' + df['Year'].astype(str)+ ' ' + df['Hour'].astype(str)+ ':' + df['Minute'].astype(str)
  df.index = pd.to_datetime(df['DATE'])
  df = df.asfreq('5T')
  df = df.interpolate(method='polynomial', order=1)

  df.to_csv(f"{DATA_DIR_SOLAR}/solar_{y}_5m.csv")
'''

In [ ]:
DATA_DIR_SOLAR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance"
solar_cols = ['Global Horizontal UV Irradiance (280-400nm)','Global Horizontal UV Irradiance (295-385nm)']
hist_window=18;forecast_window=18
i=0
for y in range(2017,2022):
  df = pd.read_csv(f"{DATA_DIR_SOLAR}/solar_{y}.csv",header=2)

  #account for previous year's last line and next year's first line for interpolation
  if i!=0:     
    df_first_line = df_last_line = pd.read_csv(f"{DATA_DIR_SOLAR}/solar_{y-1}.csv",header=2).iloc[-1] 
    df = pd.concat([df_first_line,df]).bfill().ffill()
  else: df = df.bfill()

  if i<len(range(2017,2022))-1: 
    df_last_line = pd.read_csv(f"{DATA_DIR_SOLAR}/solar_{y+1}.csv",header=2).iloc[0]
    df = pd.concat([df,df_last_line]).bfill().ffill()
  else: df = df.ffill()
  
  #print(df.head(10))

  #enforce 5 m and Interpolate
  for c in ['Month','Day','Year','Hour','Minute']:     df[c]=df[c].astype(int).astype(str)
  df['DATE'] = df['Month'].astype(str)+ '-' +df['Day'].astype(str)+ '-' + df['Year'].astype(str)+ ' ' + df['Hour'].astype(str)+ ':' + df['Minute'].astype(str)
  df['DATE'] = pd.to_datetime(df['DATE']).dt.tz_localize('utc').dt.tz_convert('US/Eastern')
  df['DATE'] = pd.to_datetime(df['DATE'].astype(str).str[0:-6])

  df.index = df['DATE']
  df = df.drop_duplicates(subset='DATE')
  df = df.asfreq('5T')
  df = df[solar_cols].interpolate(method='polynomial', order=1).bfill().ffill()

  

  i=i+1
  df.to_csv(f"{DATA_DIR_SOLAR}/solar_{y}_5m.csv")

        0  Clearsky DHI  Clearsky DNI  Clearsky GHI  Cloud Type  DHI  DNI  \
0  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
1  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
2  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
3  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
4  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
5  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
6  2018.0           0.0           0.0           0.0         8.0  0.0  0.0   
7  2018.0           0.0           0.0           0.0         4.0  0.0  0.0   
8  2018.0           0.0           0.0           0.0         4.0  0.0  0.0   
9  2018.0           0.0           0.0           0.0         1.0  0.0  0.0   

   Day  Dew Point  Fill Flag  ...  Month  Precipitable Water  Pressure  \
0  1.0        0.5        0.0  ...    1.0                 1.3    1014.0   
1  1

#Load the Weather Data

In [ ]:
DATA_DIR_WEATHER = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Weather/JFK"
weather_cols =['HourlyDryBulbTemperature','HourlyRelativeHumidity']
hist_window=18;forecast_window=18

i=0
for y in range(2017,2022):
  df = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y}.csv",header=0)
  
  #account for previous year's last line and next year's first line for interpolation
  if i!=0:     
    df_first_line = df_last_line = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y-1}.csv",header=0).iloc[-5:-1] 
    #print(df_first_line[['DATE']+weather_cols])
    #df_first_line = df_last_line = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y-1}_5m.csv",header=0).iloc[-2:-1] 
    df = pd.concat([df_first_line,df],axis=0)
  else: df = df.bfill()

  if i<len(range(2017,2022)): 
    df_last_line = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y+1}.csv",header=0).iloc[0:5]
    df = pd.concat([df,df_last_line],axis=0)
  else: df = df.ffill()


  df = df.drop_duplicates(subset='DATE')
  for c in weather_cols:
    df[c]=df[c].astype(str).str.replace('s','')
    df[c]=df[c].astype(float)

  #print(df.head(10))

  #enforce 5 m and Interpolate
  df.index = pd.to_datetime(df['DATE'])
  df = df.asfreq('1T')
  df = df.interpolate(method='polynomial', order=1).bfill().ffill()
  m = (df.index.minute % 5 == 0 ) & (df.index.year  == y)
  df[m].to_csv(f"{DATA_DIR_WEATHER}/JFK_{y}_5m.csv")
  i=i+1


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,41,64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-50-f5ebeffdd7c3>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c]=df[c].astype(str).str.replace('s','')
<ipython-input-50-f5ebeffdd7c3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

#Load the Electrical Load Data and Merge

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data"
DATA_DIR_LOAD = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Load"
DATA_DIR_SOLAR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance"
DATA_DIR_WEATHER = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Weather/JFK"

hist_window=18;forecast_window=18
#hist_window=int(12*60/5)

In [ ]:
solar_cols = ['Global Horizontal UV Irradiance (280-400nm)','Global Horizontal UV Irradiance (295-385nm)']
weather_cols =['HourlyDryBulbTemperature','HourlyRelativeHumidity']
for y in range(2017,2022):
  df_l = pd.read_csv(f"{DATA_DIR_LOAD}/Load_{y}.csv") #Time Stamp 
  df_l = df_l.set_index('Time Stamp')
  df_s = pd.read_csv(f"{DATA_DIR_SOLAR}/solar_{y}_5m.csv") #DATE
  df_s = df_s.set_index('DATE')
  df_w = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y}_5m.csv") #DATE
  df_w = df_w.set_index('DATE')

  

  df = pd.concat([df_l, df_s[solar_cols], df_w[weather_cols]], axis=1).bfill().ffill()
  df.index = pd.to_datetime(df.index)
  m=df.index.year  == y
  df[m].to_csv(f"{DATA_DIR}/Total_Data_{y}_5m.csv")


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,18,19,20,42,43,65,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,18,19,20,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,18,19,20,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,18,19,20,22,24,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_n